In [7]:
# Load preprocessed Data
import sys
sys.path.append('/home/alexabades/recsys')

import pandas as pd 
import numpy as np 
import torch 
from torch import nn, optim 
import torch
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from src.models.AutoEncoder.AE import AutoEncoder
from src.data.BinaryClassifictionDataLoader import ContextDataLoaderBinaryClasifictaion
import torch 
from torch.utils.data import DataLoader

_optimizers = {"adam": optim.Adam, "SGD": optim.SGD}
_loss_fn = {"BCE": nn.BCELoss(), "MSE": nn.MSELoss()}

In [18]:
from src.data.ContextDataLoader import ContextDataLoader


train_data_path = "../data/processed/frappeCtxA/frappeCtxA.train.rating"
test_data_path = "../data/processed/frappeCtxA/frappeCtxA.test.rating"
train_data = ContextDataLoader(train_data_path)
test_data = ContextDataLoader(test_data_path)

# Dataloader
train_loader = DataLoader(train_data, 100)
test_loader = DataLoader(test_data, 100)
epochs = 100
hidden_units=[21, 9]
ae_model = AutoEncoder(hidden_units).to(_device)


AttributeError: 'ReLU' object has no attribute 'weight'

In [19]:
ae_model

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=21, out_features=9, bias=True)
    (1): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=9, out_features=21, bias=True)
    (1): ReLU()
    (2): Sigmoid()
  )
)

In [20]:
for i in ae_model.modules():
  print(i)
  break

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=21, out_features=9, bias=True)
    (1): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=9, out_features=21, bias=True)
    (1): ReLU()
    (2): Sigmoid()
  )
)


In [46]:
noise_tensor = torch.randint(0, 2, size=(256,22), dtype=torch.float32, device=_device)
noise = torch.rand(256, 1)
noise_tensor.shape

torch.Size([256, 22])

In [43]:
noise_tensor[:, 0] = noise.squeeze()
noise_tensor

tensor([[0.0462, 1.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.2514, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.4606, 1.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        ...,
        [0.3135, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.1417, 0.0000, 1.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.6579, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],
       device='cuda:0')

In [ ]:

ae_model.train()

# Initialize Optimizer and Loss function
loss_fn = nn.MSELoss()
optimizer = optim.Adam(ae_model.parameters(), lr=0.001)

# calculate_memory_allocation()
for epoch in range(epochs):
    if not epoch % 10: 
        print(epoch)
    for batch in train_loader:
        context_input = batch["context"].to(_device)
        # TODO: Context should be floating point, at least cnt feature, what happens when is converted into long

        output = ae_model(context_input)
        loss = loss_fn(output["prediction"], context_input)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    


In [6]:
11 % 10

1

In [33]:
from src.utils.eval import mean_absolute_error, root_mean_squared_error


ae_model.eval()
with torch.no_grad():
        for batch in test_loader:
            context_input = batch["context"].to(_device)


            output = ae_model(context_input)
            # Evaluate
            rsme = root_mean_squared_error(output['prediction'], context_input)
            mae = mean_absolute_error(output['prediction'], context_input)
            break



In [35]:
rsme

0.50970936

In [10]:
import os
os.getcwd()

'/home/alexabades/recsys/src/notebooks'

In [27]:
from src.utils.model_stats.stats import save_model_with_params
chk_path = '/home/alexabades/recsys/src/notebooks/ae.bin'
model_params = {
    "hidden_dims": hidden_units,
    "dropout": 0,
    "activation": 'ReLU'
}

save_model_with_params(chk_path, ae_model, model_params)

In [28]:
def load_model_with_params(path):
    checkpoint = torch.load(path)
    model_params = checkpoint['model_params']

    # Reconstruct the model based on saved parameters
    # This assumes you have a way to map 'activation' from a string to an actual PyTorch activation function
    activation_function = getattr(nn, model_params['activation']) if 'activation' in model_params else nn.ReLU
    model = AutoEncoder(
        hidden_dims=model_params['hidden_dims'],
        dropout=model_params['dropout'],
        activation=activation_function,
    )
    
    # Load the saved state_dict into the model
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model


In [29]:
model = load_model_with_params(chk_path)

In [13]:
model.modules()

NameError: name 'model' is not defined

In [28]:
from typing import Callable, List
import warnings
from torch import nn

# TODO: Initialize mean and standard deviation


class AutoEncoder(nn.Module):
    def __init__(
        self,
        hidden_dims: List[int] = None,
        dropout: float = 0,
        activation: Callable[..., nn.Module] = nn.ReLU,
    ) -> None:
        """
        Autoencoder following an hourglass structure

        Parameters:
          - hidden_dims (List[ìnt]): List indicating the hideen dimensions of the Encoder
          - dropout (float): Droput for all hidden layers
          - activation (Callable): Activation function, default to ReLU
        """
        super().__init__()

        if not hidden_dims:
            raise ValueError("AutoEncoder initialized without dimensions")
        if len(hidden_dims) < 2:
            warnings.warn("AutoEncoder with a single hidden layer.")

        # Build the encoder layers
        encoder_layers = []
        for i in range(len(hidden_dims) - 1):
            encoder_layers.extend(
                [
                    nn.Linear(hidden_dims[i], hidden_dims[i + 1]),
                    activation(),
                    nn.Dropout(dropout),
                ]
            )

        # Build the decoder layers (reverse structure of the encoder)
        decoder_layers = []
        decoder_dims = hidden_dims[::-1]  # Reverse the dimensions for the decoder
        for i in range(len(decoder_dims) - 1):
            decoder_layers.extend(
                [
                    nn.Linear(decoder_dims[i], decoder_dims[i + 1]),
                    activation(),
                    nn.Dropout(dropout),
                ]
            )

        # Removing the last Dropout from the encoder and decoder
        encoder_layers = encoder_layers[:-1]
        decoder_layers = decoder_layers[:-1]

        self.encoder = nn.Sequential(*encoder_layers)
        self.decoder = nn.Sequential(*decoder_layers, nn.Sigmoid())

        self._init_weights()

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return {"latent": z, "prediction": x_hat}

    def _init_weights(self):
        for module in self.modules():
            # print(module)
            if isinstance(module, nn.Linear):
                print(module.weight)
                nn.init.normal_(module.weight, mean=0.0, std=0.01)
                print(module.weight)
                if module.bias is not None:
                    nn.init.constant_(
                        module.bias, 0
                    )  # Optionally initialize biases to 0

    # def _init_weights(self):
    #     for layer in self.encoder:
    #         nn.init.normal_(layer.weight, mean=0.0, std=0.01)
    #     for layer in self.decoder:
    #         nn.init.normal_(layer.weight, mean=0.0, std=0.01)

In [29]:
model = AutoEncoder([22,9])

Parameter containing:
tensor([[ 0.1380,  0.0920,  0.1286, -0.1268, -0.1782,  0.2025, -0.0457,  0.0156,
          0.1342, -0.1020,  0.1070,  0.1477, -0.1106, -0.0763, -0.2079, -0.0231,
         -0.0884, -0.0955,  0.1448, -0.1253,  0.0222, -0.0599],
        [-0.0028, -0.0679,  0.1249,  0.1438,  0.1213,  0.1215, -0.1145,  0.0598,
          0.0084,  0.0961, -0.0105,  0.1192, -0.1957,  0.0996, -0.0661, -0.1028,
          0.0415,  0.2097, -0.1590, -0.1220, -0.1909, -0.1629],
        [-0.0182,  0.1747,  0.0559,  0.2077,  0.0119,  0.1330,  0.1812,  0.1886,
         -0.0842, -0.0272, -0.0377, -0.2023,  0.0162, -0.0511,  0.1733,  0.0944,
          0.0859, -0.1093, -0.2095, -0.2083, -0.1753,  0.0903],
        [ 0.0979, -0.1212, -0.1360,  0.1611, -0.1027,  0.1105, -0.0692, -0.0717,
          0.0216,  0.0846,  0.0263,  0.1416, -0.1616, -0.1455,  0.1220,  0.1633,
         -0.1788, -0.0649,  0.1084, -0.1242,  0.2046, -0.0998],
        [-0.0326, -0.0544,  0.1637, -0.1932,  0.0180,  0.0595, -0.1286, -0

In [25]:
model.modules()

<generator object Module.modules at 0x7fb300f4c120>